# Colab 量化程式設計：多儲存格工作流

這份文件定義了在 Google Colab 中執行量化研究的標準作業流程。此設計的核心是將 Colab 視為一個純粹的「執行環境」，並將流程拆分為多個獨立的儲存格，以便於分段執行、控制和除錯。

## 使用說明

請依照以下順序，將每個儲存格的程式碼複製到您自己的 Google Colab 筆記本中，然後逐一執行。每個程式碼儲存格都包含了 `#@title` 和中文標題，方便您在 Colab 中將其摺疊收合，保持筆記本的整潔。

## 儲存格 1：掛載資料儲存庫 (Google Drive)

### 目的
建立 Colab 環境與您的 Google Drive 之間的連接，並設定好專案所需的資料夾結構。

### 執行動作
1.  **觸發授權**：執行後，會跳出一個 Google 授權視窗，請登入您的帳戶並授權 Colab 存取您的雲端硬碟。
2.  **掛載硬碟**：將您的 Google Drive 掛載到 Colab 的 `/content/drive/MyDrive` 路徑。
3.  **建立目錄**：腳本會自動檢查指定的專案根目錄 (`MyQuantProject`) 以及其中的 `data`, `models`, `results` 子目錄是否存在。如果不存在，將會自動建立。

### 產出
*   Colab 獲得對您 Google Drive 的完整讀寫權限。
*   一個標準化的專案資料夾結構，確保後續步驟可以正確地讀寫檔案。

### 複製以下程式碼至 Colab 儲存格：

In [ ]:
#@title 1. 掛載 Google Drive 並建立專案目錄
from google.colab import drive
import os

#@markdown ### **設定 Google Drive 專案目錄**
#@markdown 請輸入您希望在 Google Drive 中使用的專案根目錄名稱。
DRIVE_PROJECT_ROOT = "MyQuantProject" #@param {type:"string"}
# --- 設定區結束 ---

try:
    # 掛載 Google Drive
    print("正在嘗試掛載 Google Drive...")
    drive.mount('/content/drive', force_remount=True)
    gdrive_path = os.path.join('/content/drive/MyDrive', DRIVE_PROJECT_ROOT)
    
    # 將雲端路徑設定為環境變數
    os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path
    # 同時也將它設定為預設的專案路徑
    os.environ['PROJECT_ROOT_PATH'] = gdrive_path
    
    print(f"✅ Google Drive 掛載成功！專案雲端路徑設定為：{gdrive_path}")
    
    # 檢查並在雲端硬碟上建立所需的核心資料夾
    for folder in ['data', 'models', 'results']:
        folder_path = os.path.join(gdrive_path, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"📁 已在雲端建立資料夾：{folder_path}")
            
except Exception as e:
    # 如果掛載失敗，清除雲端路徑的環境變數，並給予提示
    os.environ['GDRIVE_PROJECT_PATH'] = ""
    print(f"⚠️ 掛載 Google Drive 失敗或未授權。錯誤：{e}")
    print("後續儲存格將使用 Colab 本機儲存空間。")

## 儲存格 2：同步程式碼 (from GitHub)

### 目的
從 GitHub 獲取最新版本的「執行邏輯」（也就是我們的 Python 腳本）。

### 執行動作
1.  **檢查本地狀態**：檢查 Colab 目前的工作區 (`/content/`) 是否已經存在專案的程式碼資料夾。
2.  **下載或更新**：
    *   如果 **不存在**：執行 `git clone`，將 GitHub 上的整個專案完整下載下來。
    *   如果 **已存在**：執行 `git pull`，只下載自上次以來在 GitHub 上的更新，確保程式碼永遠是最新版本。

### 產出
*   Colab 的 `/content/` 目錄下，擁有了一份與您 GitHub 儲存庫完全同步的最新程式碼。

### 複製以下程式碼至 Colab 儲存格：

In [ ]:
#@title 2. 從 GitHub 同步專案程式碼
import os

#@markdown ### **1. 設定 GitHub 儲存庫**
#@markdown 請填寫您的 GitHub 儲存庫網址。
GITHUB_REPO_URL = "https://github.com/hsp1234-web/qlib.git" #@param {type:"string"}

#@markdown ---
#@markdown ### **2. 輸入分支名稱**
#@markdown 請輸入您想要使用的分支名稱（例如 `main` 或 `feature/new-strategy`）。
GIT_BRANCH = "main" #@param {type:"string"}
# --- 設定區結束 ---

# --- 將設定儲存為環境變數，供後續儲存格使用 ---
os.environ['GITHUB_REPO_URL'] = GITHUB_REPO_URL
os.environ['GIT_BRANCH'] = GIT_BRANCH
# ---

# 從網址中提取專案名稱
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
local_repo_path = os.path.join('/content', repo_name)

print(f"目標 GitHub 儲存庫：{GITHUB_REPO_URL}")
print(f"預計本地路徑：{local_repo_path}")

# 檢查本地資料夾是否已存在
if os.path.exists(local_repo_path):
    print(f"專案資料夾已存在，正在切換至 '{GIT_BRANCH}' 分支並更新...")
    # 切換到該目錄
    os.chdir(local_repo_path)
    # 執行 git 指令來切換分支並拉取最新的程式碼
    get_ipython().system(f'git fetch origin')
    get_ipython().system(f'git checkout {GIT_BRANCH}')
    get_ipython().system(f'git pull origin {GIT_BRANCH}')
    # 切換回 content 根目錄
    os.chdir('/content')
else:
    print(f"專案資料夾不存在，正在從 '{GIT_BRANCH}' 分支下載整個專案...")
    # 使用 --branch 參數來 clone 指定的分支
    get_ipython().system(f'git clone --branch {GIT_BRANCH} {GITHUB_REPO_URL}')

print("\n✅ 程式碼同步完成！")
# 顯示目前資料夾結構，以確認程式碼已成功下載
print("\n--- 目前 /content/ 目錄結構 ---")
get_ipython().system('ls -F /content/')

## 儲存格 3：安裝/更新環境 (Requirements)

### 目的
確保 Colab 環境安裝了所有執行我們的量化分析腳本所需要的 Python 套件。

### 執行動作
1.  **讀取設定檔**：找到上一步從 GitHub 同步下來的 `requirements.txt` 檔案。
2.  **安裝套件**：使用 `pip install -r` 指令，自動安裝或更新清單中指定的所有 Python 函式庫。

### 產出
*   一個準備就緒、包含所有依賴項的 Python 執行環境。

### 複製以下程式碼至 Colab 儲存格：

In [ ]:
#@title 3. 安裝專案所需的 Python 套件
import os

# --- 從環境變數讀取設定 ---
# 這些設定是在儲存格 2 中定義的，確保了設定的統一性
GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
GIT_BRANCH = os.environ.get('GIT_BRANCH')

if not GITHUB_REPO_URL or not GIT_BRANCH:
    raise ValueError("錯誤：找不到 GITHUB_REPO_URL 或 GIT_BRANCH 環境變數。請先執行儲存格 2。")
# ---

# 從網址中動態推斷專案名稱
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
requirements_path = os.path.join('/content', repo_name, 'requirements.txt')

print(f"正在讀取設定檔：{requirements_path}")

if os.path.exists(requirements_path):
    print("找到 requirements.txt，開始安裝/更新環境...")
    # 使用 -q 參數來減少不必要的輸出訊息，讓介面更簡潔
    get_ipython().system(f'pip install -q -r {requirements_path}')
    print("\n✅ 環境安裝完成！")
else:
    print(f"⚠️ 警告：在 '{requirements_path}' 中找不到 requirements.txt 檔案。")
    print("請確認您的 GitHub 儲存庫中是否包含此檔案。")

## 儲存格 4：執行回測

### 目的
執行主要的量化回測任務。

### 執行動作
1.  **設定參數**：透過下方的互動式表單，設定您想要回測的股票代碼、日期範圍和交易策略。
2.  **調用腳本**：點擊執行後，系統會自動將您設定的參數組合起來，並執行 `backtest_runner.py` 腳本。
3.  **執行流程**：腳本會完成資料下載、策略回測、績效分析等所有步驟。

### 產出
*   在儲存格的輸出區塊，您將會看到詳細的回測過程日誌以及最終的績效分析報告。
*   所有詳細的交易紀錄和分析結果，都會被儲存到您在「儲存格 1」中設定的 `results` 資料夾中。

### 複製以下程式碼至 Colab 儲存格：

In [ ]:
#@title 4. 執行回測 { display-mode: "form" }
#@markdown --- 
#@markdown ### **回測參數設定**
#@markdown 請在下方輸入您想要回測的參數。

TICKER = "^TWII" #@param {type:"string"}
START_DATE = "2020-01-01" #@param {type:"date"}
END_DATE = "2023-01-01" #@param {type:"date"}
STRATEGY = "defaultlong" #@param ["defaultlong", "defaultshort", "defaultall"]

# --- 執行回測 ---
import os

# 從環境變數中讀取 GitHub repo URL
GITHUB_REPO_URL = os.environ.get('GITHUB_REPO_URL')
if not GITHUB_REPO_URL:
    raise ValueError("錯誤：找不到 GITHUB_REPO_URL 環境變數。請先執行儲存格 2。")

# 從 repo URL 推斷專案名稱
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')
script_path = f"/content/{repo_name}/src/backtest_runner.py"

# 檢查腳本是否存在
if not os.path.exists(script_path):
    raise FileNotFoundError(f"錯誤：找不到回測腳本 '{script_path}'。請確認儲存格 2 已成功執行，且您的專案中包含此檔案。")

print(f"--- 執行回測：{TICKER} ---")
command = (
    f"python {script_path} "
    f"--ticker \"{TICKER}\" "
    f"--start_date \"{START_DATE}\" "
    f"--end_date \"{END_DATE}\" "
    f"--strategy {STRATEGY}"
)

print(f"執行命令: {command}")
get_ipython().system(command)

print("\n✅ 回測執行完畢！")

## 儲存格 5：(可選) 同步本地成果至 Google Drive

### 目的
如果您在未掛載 Google Drive 的情況下執行了前面的步驟（例如儲存格 4），您的所有成果（下載的資料、回測的結果）都會被儲存在 Colab 的**本機**臨時空間中。這個儲存格提供了一個方便的功能，讓您可以**事後**將這些儲存在本機的成果，完整地同步回您的 Google Drive，以作永久保存。

### 執行動作
1.  **檢查雲端路徑**：檢查「儲存格 1」是否已經成功執行並設定了 Google Drive 的路徑。
2.  **觸發授權與掛載**：如果 Google Drive 尚未掛載，它會自動執行掛載流程，並要求您授權。
3.  **複製檔案**：使用 `distutils` 工具，遞迴地將 Colab 本機專案資料夾中的所有檔案和子資料夾，複製到您在 Google Drive 上指定的專案目錄中。

### 產出
*   您在本機執行的所有成果，都會被完整地備份一份到您的 Google Drive 中。

### 複製以下程式碼至 Colab 儲存格：

In [ ]:
#@title 5. (可選) 同步本地成果至 Google Drive
import os
from google.colab import drive
from distutils.dir_util import copy_tree

# --- 從環境變數讀取雲端路徑 ---
# 這個變數只有在儲存格 1 成功執行後才會被設定
gdrive_path = os.environ.get('GDRIVE_PROJECT_PATH')
# 獲取當前設定的（可能是本機的）專案根目錄
project_root = os.environ.get('PROJECT_ROOT_PATH')

# 檢查 project_root 是否存在
if not project_root or not os.path.exists(project_root):
    print("❌ 錯誤：找不到任何本地專案成果。請先執行儲存格 4。")
# 檢查本地路徑是否與雲端路徑相同
elif gdrive_path and project_root == gdrive_path:
    print("✅ 您的專案成果已經儲存在 Google Drive 中，無需同步。")
else:
    print("🔄 開始將本地成果同步至 Google Drive...")
    
    # 如果雲端路徑不存在 (儲存格 1 未執行或失敗)，則需要先掛載
    if not gdrive_path:
        try:
            print("正在嘗試掛載 Google Drive...")
            drive.mount('/content/drive', force_remount=True)
            
            # 從儲存格 1 的參數中讀取預設的專案名稱
            project_name = "MyQuantProject" # 確保這裡的名稱與儲存格 1 的預設值一致
            gdrive_path = os.path.join('/content/drive/MyDrive', project_name)
            os.environ['GDRIVE_PROJECT_PATH'] = gdrive_path
            print(f"✅ Google Drive 掛載成功！雲端路徑設定為：{gdrive_path}")
        except Exception as e:
            print(f"❌ 掛載 Google Drive 失敗。無法同步。錯誤：{e}")
            # 將 gdrive_path 設為 None 以跳過後續操作
            gdrive_path = None

    # 如果成功獲取到雲端路徑，則開始複製
    if gdrive_path:
        print(f"正在將 '{project_root}' 的內容複製到 '{gdrive_path}'...")
        
        # 確保雲端目標資料夾存在
        os.makedirs(gdrive_path, exist_ok=True)
        
        # 執行複製
        copy_tree(project_root, gdrive_path)
        
        print("✅ 同步完成！所有本地成果已成功複製到 Google Drive。")